# HEADER FILE

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from collections import Counter,defaultdict
import matplotlib.image as mpimg
from IPython.display import clear_output
import time
import warnings
warnings.filterwarnings('ignore')
from  sklearn.model_selection import train_test_split
import sys,os
from shutil import copyfile
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = ''


# DATA PATH 

In [2]:
attribute_img_path = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/Category and Attribute/Anno/list_attr_img.txt'
attribute_cloth_path = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/Category and Attribute/Anno/list_attr_cloth.txt'
img_path = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/Category and Attribute/img/'

# DATA VISUALISATION

In [3]:
attribute_cloth = pd.read_csv(attribute_cloth_path, sep="\s\s+")
print 'done'

done


In [4]:
attribute_cloth.head()

,attribute_name,attribute_type
0,a-line,3
1,abstract,1
2,abstract chevron,1
3,abstract chevron print,1
4,abstract diamond,1


In [5]:
chunksize = 10 ** 3 
attribute_img = 0
for chunk in pd.read_csv(attribute_img_path, sep="\s\s\s+", chunksize=chunksize):
    attribute_img = chunk
    break
attribute_img.head()

,image_name,attribute_labels
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...


# ATTRIBUTE_IMG_PATH_DETAILS

In [ ]:
#-----------------------    DATA greater than 1GB SIZE, SO READING CHUNK WISE    --------------------------
chunksize = 10 ** 4  

attribute_type = 1  # ------------ considering only class 1 - TEXTURE       

dd = []
for chunk in pd.read_csv(attribute_img_path, sep="\s\s\s+", chunksize=chunksize):
    attribute_img = chunk
    
    for i in attribute_img['attribute_labels']:
        if type(i) == str:

            ll = i.split()
            ll = np.array(ll)
            pos = np.where(ll == '1')[0]
            for j in pos:
                if attribute_cloth.iloc[j]['attribute_type'] == attribute_type: 
                    dd.append(j+1)
dist = Counter(dd)
print dist

In [ ]:
# ---------------------     KEEPING IMAGE NAME IN LIST, FOR FURTHER NEED     -----------------------
chunksize = 10 ** 4

image_name = []
for chunk in pd.read_csv(attribute_img_path, sep="\s\s\s+", chunksize=chunksize):
    attribute_img = chunk
    
    for i in attribute_img['image_name']:
        image_name.append(i)
        
print len(image_name)

In [9]:
# -------------------      JUST CONFORING ABOVE CELL OUTPUT      -----------------------------
print image_name[0]
print image_name[1]

img/Sheer_Pleated-Front_Blouse/img_00000001.jpg
img/Sheer_Pleated-Front_Blouse/img_00000002.jpg


# ATTRIBUTE_CLOTH_PATH_DETAILS

In [ ]:
#---------------------      ROOT CLASS DISTRIBUTION                ---------------------

# CONVERT 1000 CLASS TO THEIR NAME
order_map = {}
for i in attribute_cloth.index:
    if attribute_cloth.iloc[i]['attribute_type'] == attribute_type: 
        order_map[i+1] = attribute_cloth.iloc[i]['attribute_name']

    
# print order_map
print 
#   DISTRIBUTION
attribute_cloth_map = defaultdict()
    
for i in attribute_cloth.index:
    if attribute_cloth.iloc[i]['attribute_type'] == attribute_type: 
        attribute_cloth_map[i+1] = attribute_cloth.loc[i]['attribute_type']
    
print 'done'
print attribute_cloth_map
    
# class1000_map = {}

In [ ]:
# ## --------------  VISUALIZATION -------------------

# for i in class1000_map.keys():
#     clear_output( wait = True )
    
#     img = mpimg.imread(img_path+i)
#     plt.imshow(img)
#     plt.show()
#     print "Class : "+str(class50_map[i])
#     time.sleep(1)

## DATASET PREPARATION : FILTERING MINORITY CLASSES

In [ ]:
## ===========      CREATE A DATAFRAME OF IMAGE PATH AND ITS LABELS   =====================

image_path = []
image_label = []

loc = 0
for chunk in pd.read_csv(attribute_img_path, sep="\s\s\s+", chunksize=chunksize):
    attribute_img = chunk
    
    
    for i in attribute_img['attribute_labels']:
        if type(i) == str:

            ll = i.split()
            ll = np.array(ll)
            pos = np.where(ll == '1')[0]
            for j in pos:
                if attribute_cloth.iloc[j]['attribute_type'] == attribute_type:
                    if dist[j+1]>=7559:
                        image_path.append(image_name[loc])
                        image_label.append(order_map[j+1])
                        #class1000_map[image_name[loc]] = order_map[j]
        loc += 1
            
print 'done'
print loc

In [ ]:
# --------------------------- CREATING DATAFRAME AFTER REMOVING MINORITY CLASS ---------------

df = pd.DataFrame({'path':image_path,'label':image_label})
df.to_csv('dataset.csv')

print "DATASET CLASS DISTRIBUTION\n"
print "n_classes : "+str(len(Counter(df['label'])))
print Counter(df['label'])

# TOP 5 ATTRIBUTE VISUALISATION

In [ ]:
import seaborn as sb

# gen_order = df['label'].value_count().index
sb.countplot(data=df, x='label', color='blue')
# plt.bar(df, label=df['label'], color='blue')
# plt.legend()
plt.ylabel('#IMAGES')
plt.xticks(rotation=45)
plt.title('Top-5 Attribute of TEXTURE')
plt.savefig('texture.png')
plt.show()

In [13]:
# ------------------       DATAFRAME OUTPUT ---------------------------------
df.head(3)

,label,path
0,print,img/Sheer_Pleated-Front_Blouse/img_00000016.jpg
1,floral,img/Sheer_Pleated-Front_Blouse/img_00000035.jpg
2,striped,img/Sheer_Pleated-Front_Blouse/img_00000054.jpg


# CLASS BALANCE

In [26]:
## balance classes
UPPER_LIMIT = 7000#9000

count_dict = defaultdict(lambda : 0)

labels =[]
paths =[]

for i,l in enumerate(image_label):
    if l=='stripe' or l =='floral' or l =='print' or l=='striped' or l=='floral print':
        if count_dict[l] < UPPER_LIMIT:
            labels.append(image_label[i])
            paths.append(image_path[i])
    count_dict[l]+=1
    
df = pd.DataFrame({'path':paths,'label':labels})
Counter(df['label'])

Counter({'floral': 7000,
         'floral print': 7000,
         'print': 7000,
         'stripe': 7000,
         'striped': 7000})

# DATA SPLIT

In [27]:
## PREPARE   TRAIN , TEST , VALIDATION :STRATIFIED
TEST_SIZE = 0.999

X_train, X_test, y_train, y_test = train_test_split(df['path'], df['label'],stratify=df['label'], test_size=TEST_SIZE)

train = pd.DataFrame({'path':X_train,'label':y_train})
test_data = pd.DataFrame({'path':X_test,'label':y_test})

# clear variables

del X_train, X_test, y_train, y_test
print "TRAIN data size "
print train.shape

print "TEST and VALIDATION data size  "
print test_data.shape

X_val, X_test, y_val, y_test = train_test_split(test_data['path'], test_data['label'],stratify=test_data['label'], test_size=0.25)

val = pd.DataFrame({'path':X_val,'label':y_val})
test = pd.DataFrame({'path':X_test,'label':y_test})

# clear variables

del X_val, X_test, y_val, y_test,df

Counter(test['label'])


#  ====== DROP INDEXES ============


val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

TRAIN data size 
(35, 2)
TEST and VALIDATION data size  
(34965, 2)


In [31]:
print sum(Counter(test['label']).values())
print sum(Counter(val['label']).values())

8742
26223


In [32]:
test.head()

,label,path
0,striped,img/Classic_Striped_Tee/img_00000047.jpg
1,floral,img/Bell-Sleeved_Lace_Crop_Top/img_00000069.jpg
2,floral print,img/Floral_Mosaic_Print_Jumpsuit/img_00000015.jpg
3,striped,img/Ruffled_Trim_Sweater/img_00000050.jpg
4,floral,img/Floral_Chiffon_Blouse/img_00000149.jpg


In [33]:
test.head()

,label,path
0,striped,img/Classic_Striped_Tee/img_00000047.jpg
1,floral,img/Bell-Sleeved_Lace_Crop_Top/img_00000069.jpg
2,floral print,img/Floral_Mosaic_Print_Jumpsuit/img_00000015.jpg
3,striped,img/Ruffled_Trim_Sweater/img_00000050.jpg
4,floral,img/Floral_Chiffon_Blouse/img_00000149.jpg


# CREATE DIRECTORIES AND SAVE DATA

In [34]:
#-----------CREATE DICRETORIES AND SAVE DATA ----------------------------
# dest_dir = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/small_data_attri1/test/'
dest_dir = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/30_11_new/small_data_attri1/test/'
def create_folders(data):
    
    print "Total Images : "+str(data.shape[0])
    global dest_dir
    for i in data.index:
        label = data.iloc[i]['label']
        src = img_path+data.iloc[i]['path']
        
#         if not os.path.exists(dest_dir+label):
#             os.makedirs(dest_dir+label)
        copyfile(src, dest_dir+label+'/'+data.iloc[i]['path'].split('/')[-2]+'_'+data.iloc[i]['path'].split('/')[-1])
        if i%1000==0:
            print i
    
create_folders(test)    

Total Images : 8742
0
1000
2000
3000
4000
5000
6000
7000
8000


In [35]:
#-----------CREATE DICRETORIES AND SAVE DATA ----------------------------
# dest_dir = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/small_data_attri1/val/
dest_dir = '/media/mint/New Volume/3_semester_IIIT_D/CF/project/30_11_new/small_data_attri1/val/'
def create_folders(data):
    
    print "Total Images : "+str(data.shape[0])
    global dest_dir
    for i in data.index:
        label = data.iloc[i]['label']
        src = img_path+data.iloc[i]['path']
        
#         if not os.path.exists(dest_dir+label):
#             os.makedirs(dest_dir+label)
            
        copyfile(src, dest_dir+label+'/'+data.iloc[i]['path'].split('/')[-2]+'_'+data.iloc[i]['path'].split('/')[-1])
        if i%1000==0:
            print i
    
create_folders(val)    

Total Images : 26223
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
